***카테고리 딕션어리***

In [ ]:
main_cat_dict = {
    '스킨케어' : '10000010001',
    '바디케어' : '10000010003',
    '헤어케어' : '10000010004',
    '미용소품' : '10000010006',
    '남성' : '10000010007',
    '마스크팩' : '10000010009',
    '클렌징' : '10000010010',
    '선케어' : '10000010011',
    '여성/위생용품': '10000020004',
}

sub_cat_dict = {
    # 스킨케어
    '토너/로션/올인원' : '100000100010008',
    '에센스/크림' : '100000100010009',
    '미스트/오일' : '100000100010010',
    
    # 바디케어
    '샤워/입욕' : '100000100030005',
    '로션/오일' : '100000100030014',
    '바디미스트' : '100000100030015',
    
    # 헤어케어
    '트리트먼트/팩' : '100000100040007',
    '샴푸/린스' : '100000100040008',

    # 미용소품
    '스킨케어소품' : '100000100060006',
    
    # 남성
    '스킨케어' : '100000100070007',
    '헤어케어' : '100000100070009',
    '쉐이빙' : '100000100070010',
    '바디케어' : '100000100070011',
    '향수/매너용품' : '100000100070017',
    
    # 마스크팩
    '시트팩' : '100000100090001',
    '페이셜팩' : '100000100090002',
    '코팩/패치' : '100000100090003',
    '패드' : '100000100090004',
    
    # 클렌징
    '클렌징폼/젤' : '100000100100001',
    '오일/워터/리무버' : '100000100100002',
    '필랭/패드' : '100000100100003',
    
    # 선케어
    '선블록' : '100000100110001',
    '태닝/애프터선' : '100000100110002',
    
    # 여성/위생용품
    '여성청결제' : '100000200040002'
    
}


# 닥터지 카테고리 
category_dict_01 = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009'  # 에센스/크림
    ]
}


***상품 넘버 가져오기***

***brand_dict, final_dict만 활용***

In [7]:
import pandas as pd
import numpy as np
import requests
import json
import time

category_dict = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009'  # 에센스/크림
    ],
    '10000010007': [ # 남성
        '100000100070007', # 스킨케어
        '100000100070009', # 헤어케어
        '100000100070010', # 쉐이빙
        '100000100070011', # 바디케어
        '100000100070017' # 향수/매너용품
    ],
    '10000010010' : [ # 클렌징
        '100000100100001', # 클렌징폼/젤
        '100000100100002', # 오일/워터/리무버
        '100000100100003' # 필링/패드
    ],
    '10000010009' : [ # 마스크팩
        '100000100090001', # 시트팩
        '100000100090002', # 페이셜팩
        '100000100090003', # 코팩/패치
        '100000100090004', # 패드
    ]
}

def bring_brand_items():
    brand_list = ['닥터지', '라운드랩', '라네즈', '아이소이', '아누아', '에스트라'] # 브랜드 이름 추가 및 
#     final_dict = {}
    item_list = list()
    for brand in brand_list:
        for main, subs in category_dict.items():
            for sub in subs:
                try: # try/except를 하는 이유는 category_dict에 있는 메인 혹은 서브 카테고리가 해당 브랜드에 없을 수도 있기 때문에
                     # 없는 경우 넘어가주도록 합니다. 
                    startCount = 0
                    while True:
                        base_url = 'https://www.oliveyoung.co.kr/store/search/NewMainSearchApi.do?'
                        params_01 = {
                            'query' : brand, # 브랜드별
                            'collection': 'OLIVE_GOODS,OLIVE_PLAN,OLIVE_EVENT,OLIVE_BRAND',
                            'listnum': '36',      
                            'startCount' : startCount, # listnum이 36이 넘어가는 경우 그 다음 페이지도 가져올 수 있도록
                            'sort' : 'RNK/DESC',
                            'goods_sort' : 'WEIGHT/DESC,RNK/DESC',
                            'disPlayCateId' : main,
                            'cateId': main, # main category
                            'cateId2': sub, # sub category
                            'typeChk' : 'thum',
                            'quickYn' : 'N'
                        }

                        headers = { 'X-Requested-With' : 'XMLHttpRequest'}
                        res = requests.get(base_url, headers = headers, params=params_01)
                        crawled_data = json.loads(res.json())

                        goodsNo_list = list() # 다음페이지의 상품 리스트가 있는지 없는지 확인하기 위한 리스트 
#                         brand_dict = {}
                        for data in crawled_data['Data'][0]['Result']: # 'Result' key부분에 상품 번호가 있습니다. 
                            goodsNo_list.append(data['GOODS_NO']) 
#                         brand_dict[sub] = goodsNo_list # brand_dict에 먼저 넣어주고
#                         final_dict[brand] = brand_dict # 다음에 final_dict에 넣어줍니다. 
                            item_list.append(data['GOODS_NO']+'_'+brand+'_'+main+'_'+sub)

                        startCount += int(params_01['listnum']) # startCount를 listnum만큼 넣어주고 다음 페이지로 이동합니다
                        if len(goodsNo_list) <= int(params_01['listnum']): # 다음 페이지에 listnum만큼 없는 경우 break를 하고 다음으로 넘어갑니다.
                            break
                except: # 메인 혹은 서브 카테고리가 없는 경우 넘어가주기 
                    continue

                    time.sleep(0.3) # 혹시나 블락을 당할 경우를 고려하여 time.sleep()을 줍니다. 

    return item_list

b_list = bring_brand_items()

0      [A000000182369, 닥터지, 10000010001, 100000100010...
1      [A000000174023, 닥터지, 10000010001, 100000100010...
2      [A000000171651, 닥터지, 10000010001, 100000100010...
3      [A000000163655, 닥터지, 10000010001, 100000100010...
4      [A000000165624, 닥터지, 10000010001, 100000100010...
                             ...                        
305    [A000000168448, 에스트라, 10000010010, 10000010010...
306    [A000000139940, 에스트라, 10000010010, 10000010010...
307    [A000000129026, 에스트라, 10000010010, 10000010010...
308    [A000000162948, 에스트라, 10000010009, 10000010009...
309    [A000000169067, 에스트라, 10000010009, 10000010009...
Length: 310, dtype: object

***sub_dict, main_dict, final_dict 활용***

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import requests
from bs4 import BeautifulSoup
import json
import time

# category_dict = {
#     '10000010001' : [ # 스킨케어
#         '100000100010008', # 토너/로션/올인원
#         '100000100010009', # 에센스/크림
#         '100000100010010' # 미스트/오일
#     ],
#     '10000010007': [ # 남성
#         '100000100070007', # 스킨케어
#         '100000100070009', # 헤어케어
#         '100000100070010', # 쉐이빙
#         '100000100070011', # 바디케어
#         '100000100070017' # 향수/매너용품
#     ],
#     '10000010010' : [ # 클렌징
#         '100000100100001', # 클렌징폼/젤
#         '100000100100002', # 오일/워터/리무버
#         '100000100100003' # 필링/패드
#     ],
#     '10000010009' : [ # 마스크팩
#         '100000100090001', # 시트팩
#         '100000100090002', # 페이셜팩
#         '100000100090003', # 코팩/패치
#         '100000100090004', # 패드
#     ]
# }

skincare_dict = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009', # 에센스/크림
        '100000100010010'
    ]
}

def bring_brand_category_dictionary():
    brand_list = ['닥터지', '라운드랩', '라네즈', '아이소이', '아누아', '에스트라'] # 브랜드 이름
    sub_dict = {}
    main_dict = {}
    final_dict = {}
    for brand in brand_list:
        for main, subs in skincare_dict.items():
            for sub in subs:
                try: # try/except를 하는 이유는 category_dict에 있는 메인 혹은 서브 카테고리가 해당 브랜드에 없을 수도 있기 때문에
                     # 없는 경우 넘어가주도록 합니다. 
                    startCount = 0
                    while True:
                        base_url = 'https://www.oliveyoung.co.kr/store/search/NewMainSearchApi.do?'
                        params_01 = {
                            'query' : brand, # 브랜드별
                            'collection': 'OLIVE_GOODS,OLIVE_PLAN,OLIVE_EVENT,OLIVE_BRAND',
                            'listnum': '36',      
                            'startCount' : startCount, # listnum이 36이 넘어가는 경우 그 다음 페이지도 가져올 수 있도록
                            'sort' : 'RNK/DESC',
                            'goods_sort' : 'WEIGHT/DESC,RNK/DESC',
                            'disPlayCateId' : main,
                            'cateId': main, # main category
                            'cateId2': sub, # sub category
                            'typeChk' : 'thum',
                            'quickYn' : 'N'
                        }

                        headers = { 'X-Requested-With' : 'XMLHttpRequest'}
                        res = requests.get(base_url, headers = headers, params=params_01)
                        crawled_data = json.loads(res.json())

                        goodsNo_list = list() # 다음페이지의 상품 리스트가 있는지 없는지 확인하기 위한 리스트 
                        tmp_dict = {}
                        for data in crawled_data['Data'][0]['Result']: # 'Result' key부분에 상품 번호가 있습니다. 
                            goodsNo_list.append(data['GOODS_NO']) 
                        sub_dict[sub] = goodsNo_list # sub_dict에 먼저 넣어주고
                        main_dict[main] = sub_dict # 그 다음에 main_dict에 넣어주고
                        final_dict[brand] = main_dict # 마지막으로 final_dict에 넣어줍니다. 

                        startCount += int(params_01['listnum']) # startCount를 listnum만큼 넣어주고 다음 페이지로 이동합니다
                        if len(goodsNo_list) <= int(params_01['listnum']): # 다음 페이지에 listnum만큼 없는 경우 break를 하고 다음으로 넘어갑니다.
                            break
                except: # 메인 혹은 서브 카테고리가 없는 경우 넘어가주기 
                    continue

                    time.sleep(0.3) # 혹시나 블락을 당할 경우를 고려하여 time.sleep()을 줍니다. 

    return final_dict

bring_brand_category_dictionary()

In [88]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os

skincare_dict = {
    '10000010001' : [ # 스킨케어
        '100000100010008', # 토너/로션/올인원
        '100000100010009', # 에센스/크림
        '100000100010010'
    ]
}


def bring_brand_items_dict():
    global skincare_dict
    brand_list = ['닥터지', '라운드랩', '라네즈', '아이소이', '아누아', '에스트라'] # 브랜드 이름 추가 및 
    brand_subcat_item_dict = {
        '닥터지' : {},
        '라운드랩' : {},
        '라네즈' : {},
        '아이소이' : {},
        '아누아' : {},
        '에스트라' : {}
    }
    for brand in brand_list:
        for main, subs in skincare_dict.items():
            for sub in subs:
                try: # try/except를 하는 이유는 category_dict에 있는 메인 혹은 서브 카테고리가 해당 브랜드에 없을 수도 있기 때문에
                     # 없는 경우 넘어가주도록 합니다. 
                    startCount = 0
                    while True:
                        base_url = 'https://www.oliveyoung.co.kr/store/search/NewMainSearchApi.do?'
                        params_01 = {
                            'query' : brand, # 브랜드별
                            'collection': 'OLIVE_GOODS,OLIVE_PLAN,OLIVE_EVENT,OLIVE_BRAND',
                            'listnum': '36',      
                            'startCount' : startCount, # listnum이 36이 넘어가는 경우 그 다음 페이지도 가져올 수 있도록
                            'sort' : 'RNK/DESC',
                            'goods_sort' : 'WEIGHT/DESC,RNK/DESC',
                            'disPlayCateId' : main,
                            'cateId': main, # main category
                            'cateId2': sub, # sub category
                            'typeChk' : 'thum',
                            'quickYn' : 'N'
                        }

                        headers = { 'X-Requested-With' : 'XMLHttpRequest'}
                        res = requests.get(base_url, headers = headers, params=params_01)
                        crawled_data = json.loads(res.json())

                        goodsNo_list = list() # 다음페이지의 상품 리스트가 있는지 없는지 확인하기 위한 리스트 
                        item_list = list()
                        for data in crawled_data['Data'][0]['Result']: # 'Result' key부분에 상품 번호가 있습니다. 
                            goodsNo_list.append(data['GOODS_NO']) 
                        item_list = goodsNo_list # brand_dict에 먼저 넣어주고
                        brand_subcat_item_dict[brand][sub] = item_list

                        startCount += int(params_01['listnum']) # startCount를 listnum만큼 넣어주고 다음 페이지로 이동합니다
                        if len(goodsNo_list) <= int(params_01['listnum']): # 다음 페이지에 listnum만큼 없는 경우 break를 하고 다음으로 넘어갑니다.
                            break
                except: # 메인 혹은 서브 카테고리가 없는 경우 넘어가주기 
                    continue

                    time.sleep(0.3) # 혹시나 블락을 당할 경우를 고려하여 time.sleep()을 줍니다.
                    
    file_path = "./brand_subcat_item_dict.json"
    with open(file_path, 'w', encoding='utf-8-sig') as file:
        json.dump(brand_subcat_item_dict, file, indent='\t')

    return 

if __name__ == '__main__':
    bring_brand_items_dict()



{'닥터지': {'100000100010008': ['A000000182369',
   'A000000174023',
   'A000000171651',
   'A000000163655',
   'A000000165624',
   'A000000179655',
   'A000000131849',
   'A000000182277',
   'A000000160395',
   'A000000154177',
   'A000000145578',
   'A000000170518',
   'A000000170038',
   'A000000165623',
   'A000000163656',
   'A000000149399',
   'A000000135930',
   'A000000140753',
   'A000000165710',
   'A000000166642',
   'A000000123503'],
  '100000100010009': ['A000000164615',
   'A000000174203',
   'A000000182586',
   'A000000151512',
   'A000000165621',
   'A000000171844',
   'A000000180436',
   'A000000172453',
   'A000000123505',
   'A000000170037',
   'A000000163653',
   'A000000163651',
   'A000000163440',
   'A000000152618',
   'A000000165711',
   'A000000174312',
   'A000000166472',
   'A000000147130',
   'A000000142567',
   'A000000180435',
   'A000000175772',
   'A000000154173',
   'A000000165694',
   'A000000147299',
   'A000000151991',
   'A000000129094',
   'A000000157

In [91]:
file_path='./test.json'

with open(file_path, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)
    
data

{'닥터지': {'100000100010008': ['A000000182369',
   'A000000174023',
   'A000000171651',
   'A000000163655',
   'A000000165624',
   'A000000179655',
   'A000000131849',
   'A000000182277',
   'A000000160395',
   'A000000154177',
   'A000000145578',
   'A000000170518',
   'A000000170038',
   'A000000165623',
   'A000000163656',
   'A000000149399',
   'A000000135930',
   'A000000140753',
   'A000000165710',
   'A000000166642',
   'A000000123503'],
  '100000100010009': ['A000000164615',
   'A000000174203',
   'A000000182586',
   'A000000151512',
   'A000000165621',
   'A000000171844',
   'A000000180436',
   'A000000172453',
   'A000000123505',
   'A000000170037',
   'A000000163653',
   'A000000163651',
   'A000000163440',
   'A000000152618',
   'A000000165711',
   'A000000174312',
   'A000000166472',
   'A000000147130',
   'A000000142567',
   'A000000180435',
   'A000000175772',
   'A000000154173',
   'A000000165694',
   'A000000147299',
   'A000000151991',
   'A000000129094',
   'A000000157

In [3]:
import os
os.getcwd()

'/Users/wonbinchoi/semiPJT_5/wonbin'

***리뷰 뽑아오는 코드***

In [ ]:
import time


review_dict = {
    'goodsNo' : [],
    'gdasCont' : [],
    'goodsNm' : [],
    'dispRegDate' : [],
    
}
# review_list = []

base_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?'
headers = {'X-Requested-With' : 'XMLHttpRequest'}
params = {
    'goodsNo' : 'A000000165683', # 상품 사전 가져오기 ==> 여기 부분 바꾸고
    'gdasSort': '02', # 그대로 놔두기
    'itemNo' : 'all_search',
    'pageIdx' : '1', # 페이지 반복문 필요 ==> 여기 부분 바꾸고 
    'cTypeLength' : '0'
}

res = requests.get(base_url, headers = headers, params=params)
crawled_data = json.loads(res.text)
for i in range(len(crawled_data['gdasList'])):
#     review_list.append(crawled_data['gdasList'][i]['gdasCont'])
    review_dict['goodsNo'].append(crawled_data['gdasList'][i]['goodsNo']) # 상품번호
    review_dict['gdasCont'].append(crawled_data['gdasList'][i]['gdasCont']) # 상품리뷰
    review_dict['goodsNm'].append(crawled_data['gdasList'][i]['goodsNm']) # 상품이름
    review_dict['dispRegDate'].append(crawled_data['gdasList'][i]['dispRegDate']) # 리뷰날짜 

# pd.DataFrame(review_dict)
crawled_data['gdasList'][0]
        